# En construcción

Load model

In [1]:
# import os

try:
  import accelerate
except ImportError:
  # instala paquetes
  %pip install -q accelerate bitsandbytes rich transformers

  print("instaladas librerías necesarias")


from transformers import AutoTokenizer, AutoModelForCausalLM

# Inicializa la variable model como None al inicio
model = None
tokenizer = None


# Función para cargar el modelo si aún no está cargado
def load_model():
    global model
    global tokenizer
    if model is None or not hasattr(model, 'num_parameters'):  # Verifica si model está vacío o no parece ser un modelo válido
        print("Cargando modelo...")
        # modelo sin cuantizar (se queda sin memoria con contexto grande)
        tokenizer = AutoTokenizer.from_pretrained("argilla/notus-7b-v1", torch_dtype="auto", trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained("argilla/notus-7b-v1", device_map='auto', torch_dtype="auto", trust_remote_code=True)
        #torch_dtype=torch.float16  (half precision) or torch.float32 (single precision que sería absurdo porque deepseek coder viene de llama 2 cuyo parámetros son float16)
        # tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True)
        # model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", device_map='auto', load_in_8bit=True, trust_remote_code=True)
        print("Modelo cargado.")
    else:
        print("Modelo ya estaba cargado.")


load_model()

Cargando modelo...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Modelo cargado.


In [3]:
    # codificar "</s>" para poder buscarlo en el texto generado
fin_generado = tokenizer.encode("</s>", add_special_tokens=False)[0]
fin_generado

2

Widget

In [2]:
import re
from IPython.display import display, HTML

from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from IPython.display import display, Image, clear_output#, Markdown
# from rich.markdown import Markdown
from IPython.display import Markdown
# import markdown

import ipywidgets as widgets

historico = ""

input_text = ""


text_input = widgets.Textarea(
    value='',
    placeholder='Escribe algo aquí, por ejemplo: /help',
    description='Input:',
    disabled=False,
    layout=widgets.Layout(width='800px', height='100px'), # Ajusta el tamaño aquí
    id='widget-textarea-id'
)


button = widgets.Button(
    description='Enviar',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' o ''
    tooltip='Enviar',
    icon='check' # (FontAwesome names sin el prefijo `fa-`)
)

def print_wrapped(text, width=120):
    words = text.split()
    line = ''

    for word in words:
        if len(line) + len(word) + 1 > width:
            print(line)
            line = word
        else:
            if line:
                line += ' '
            line += word
    print(line)

def display_response(text, image_path=None):
    # text = text.replace("�", "ú") #error de este modelo con las "ú"
    # text = text.replace("<|EOT|>", "") #no mostrar este caracter especial
    # # display(Markdown(text))  # Para texto
    # display_formatted_text(text)  # Para texto con formato
    print_wrapped(text)
    # Para imágenes (si response contiene una ruta de imagen o URL)
    # Mostrar imagen si la ruta está proporcionada
    if image_path:
        display(Image(filename=image_path))


# VENTANA DESLIZANTE
def ajustar_contexto(texto, max_longitud=4000, secuencia="### Instruction"):
    # Comprobar si la longitud del texto es mayor que el máximo permitido
    if len(texto) > max_longitud:
        indice_secuencia = 0

        while True:
            # Buscar la secuencia de ajuste
            indice_secuencia = texto.find(secuencia, indice_secuencia + 1)

            # Si la secuencia no se encuentra o el texto restante es menor que la longitud máxima
            if indice_secuencia == -1 or len(texto) - indice_secuencia <= max_longitud:
                break

        # Si encontramos una secuencia válida
        if indice_secuencia != -1:
            return texto[indice_secuencia:]
        else:
            # Si no se encuentra ninguna secuencia adecuada, tomar los últimos max_longitud caracteres
            return texto[-max_longitud:]
    else:
        return texto


def eliminar_ultima_pregunta_respuesta(texto, secuencia="### Instruction"):
    # Buscar la secuencia de ajuste
    indice_secuencia = texto.rfind(secuencia)

    # Si la secuencia no se encuentra
    if indice_secuencia == -1:
        return texto
    else:
        return texto[:indice_secuencia]


def eliminar_ultimas_preguntas_respuestas(texto, n=1, secuencia="### Instruction"):
    for i in range(n):
        texto = eliminar_ultima_pregunta_respstartswithuesta(texto, secuencia)
    return texto

# recuperar última pregunta desde ### Instruction hasta ### Response
def recuperar_ultima_pregunta(texto, secuencia="### Instruction", fin="### Response"):
    # Buscar la secuencia de ajuste
    indice_secuencia = texto.rfind(secuencia)
    indice_fin = texto.rfind(fin)

    # Si la secuencia no se encuentra
    if indice_secuencia == -1:
        return texto
    else:
        indice_secuencia += len(secuencia) + 2
        return texto[indice_secuencia:indice_fin]


def generate_long_chat(contexto, input_text, max_additional_tokens=2000):
    global historico
    global tokenizer
    global model

    prompt = f"<|user|>\n{input_text}</s>\n<|assistant|>\n"
    # streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True) # para streamear el output pero sin repetir el prompt ni el contexto anterior.

    # custom_streamer = CustomTextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)


    final_prompt = contexto + "\n" + prompt
    longitud_prompt_tokens = len(tokenizer.encode(final_prompt))

    inputs = tokenizer(final_prompt, return_tensors="pt", add_special_tokens=False)

    model_inputs = inputs.to(model.device)      # .to("cuda")
    outputs = model.generate(**model_inputs,
                            streamer=streamer,
                            max_new_tokens=max_additional_tokens,
                            #  max_length=max_length,
                            temperature=0.3,
                            top_k=50,
                            top_p=0.5,
                            pad_token_id = 2,
                            # eos_token_id=32021,
                            do_sample=True                            
                            )

    # codificar "</s>" para poder buscarlo en el texto generado
    # fin_generado = tokenizer.encode("</s>", add_special_tokens=False)[0]

    inicio_generado = longitud_prompt_tokens - 1
    decoded_output = tokenizer.decode(outputs[0][inicio_generado:], skip_special_tokens=True)

    # decoded_output = decoded_output.replace("�", "ú") #error de este modelo con las "ú"

    historico += prompt + decoded_output

    text = final_prompt + decoded_output + "</s>"
    return text



load_model()

system_prompt = """
You are an helpfull assistant.
"""
saludo = "I am an helpfull assistant. How can I help you?"

import sys
import os
import time


contexto = f"<|system|>{system_prompt}</s>\n<|assistant|>\n{saludo}</s>\n"
historico = contexto



def guardar_historico(historico, nombre_fichero):
    with open(nombre_fichero, "w", encoding="utf-8") as archivo:
        archivo.write(historico)


def on_button_clicked(b):
    global contexto, historico, input_text
    # Cambiar el estilo del botón a "procesando"
    button.description = 'Procesando...'
    button.button_style = 'warning'  # Color amarillo para indicar procesamiento
    button.disabled = True

    # Acción a realizar cuando se hace clic en el botón
    try:
        with output:
            # output.clear_output()
            print(f"User: {text_input.value}")
            input_text = text_input.value
            text_input.value = ""
            # input_text = input("user: ")
            if input_text == "/new":
                guardar_historico(historico, "last_session.txt")
                historico = ""
                contexto = system_prompt
                clear_output(wait=True)
                display_response(contexto)
            elif input_text == "/historico":
                clear_output(wait=True)
                display_response(historico)
            elif input_text == "/contexto":
                clear_output(wait=True)
                display_response(contexto)
            elif input_text.startswith("/save"):
                partes = input_text.split(maxsplit=1)
                nombre_fichero = partes[1] if len(partes) > 1 else time.strftime("%Y-%m-%d_%H-%M") + ".txt"
                guardar_historico(historico, nombre_fichero)
                display_response(f"Histórico guardado en '{nombre_fichero}'")
            elif input_text == "/len":
                display_response("longitud del contexto en caracteres: " + str(len(contexto)))
            elif input_text.startswith("/del"):
                partes = input_text.split()
                if len(partes) == 2 and partes[1].isdigit():
                    try:
                        n = int(partes[1])
                    except ValueError:
                        n = 1
                else:
                    n = 1  # Por defecto, eliminar una respuesta
                historico+= f"\n#############################\n/del {n}\n##########################################\n"
                contexto = eliminar_ultimas_preguntas_respuestas(contexto, n)
                clear_output(wait=True)
                display_response(contexto)
            elif input_text == "/repeat":
                historico+= "\n#############################\n/repeat\n##########################################\n"
                ultima_pregunta = recuperar_ultima_pregunta(contexto)
                contexto = eliminar_ultima_pregunta_respuesta(contexto)
                # imprimir ultimas 10 letras del contexto:
                print("GENERANDO STREAMING... (AUN SIN FORMATO)")
                contexto = generate_long_chat(contexto, input_text=ultima_pregunta, max_additional_tokens=2048)
                contexto = ajustar_contexto(contexto)
                clear_output(wait=True)
                display_response(contexto)
            elif input_text.startswith("/help") or input_text.startswith("/?"):
                print("""
                /new: Nuevo Chat
                /historico: mostrar el historico completo (no solo el alcance del contexto)
                /contexto: muestra el contexto de la conversación (la zona delimitada que tendrá en cuenta el modelo)
                /save [file_name]: guarda el historico conversacional en un fichero.
                /len: mostrar la longitud del contexto
                /del [n]: eliminar las últimas n respuestas
                /repeat: repetir la última respuesta
                /clear: borrar el contexto
                """)
            elif input_text == "/clear":
                historico+= "\n#############################\n/clear\n##########################################\n"
                contexto = system_prompt
                clear_output(wait=True)
                display_response(contexto)
            else:
            # generate response
                print("GENERANDO STREAMING... (AUN SIN FORMATO)")
                contexto = generate_long_chat(contexto, input_text=input_text, max_additional_tokens=2048)
                contexto = ajustar_contexto(contexto)
                clear_output(wait=True)
                display_response(contexto)
    finally:
        # Cambiar el estilo del botón de vuelta a su estado normal
        button.description = 'Enviar'
        button.button_style = 'success'  # Color verde para indicar listo
        button.disabled = False

# Crear un output para mostrar los resultados
output = widgets.Output()

# Asignar la función de callback al evento de clic del botón
button.on_click(on_button_clicked)

# Mostrar el output
display(output)


import ipywidgets as widgets
from IPython.display import display

display(widgets.HBox([text_input, button]))

Modelo ya estaba cargado.


Output()

# Pruebas